In [1]:
!pip install ultralytics --upgrade



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import cv2
import numpy as np
import time
import warnings
warnings.filterwarnings('ignore')

class VideoPersonCounter:
    def __init__(self, model_size='n'):
        """Initialize YOLO model for person counting"""
        print(f"🚀 Initializing Video Person Counter...")
        
        self.model = None
        self.model_name = "None"
        self.people_counts = []
        self.frame_count = 0
        
        # Load YOLO model
        self.load_model(model_size)
        
    def load_model(self, model_size):
        """Load YOLO model with error handling"""
        try:
            from ultralytics import YOLO
            
            # Try different model options
            model_options = [f'yolo11{model_size}.pt', f'yolov8{model_size}.pt', 'yolo11n.pt', 'yolov8n.pt']
            
            for model_path in model_options:
                try:
                    print(f"📥 Trying to load {model_path}...")
                    self.model = YOLO(model_path)
                    self.model.conf = 0.5       # Confidence threshold
                    self.model.iou = 0.45       # IoU threshold
                    self.model.max_det = 1000   # Max detections
                    self.model_name = model_path
                    print(f"✅ Successfully loaded {model_path}")
                    return
                except Exception as e:
                    print(f"⚠️ Failed to load {model_path}: {str(e)[:50]}...")
                    continue
            
            print("❌ All YOLO models failed to load")
            
        except ImportError:
            print("❌ Ultralytics not installed. Run: pip install ultralytics")
    
    def count_people_in_frame(self, frame):
        """Count people in a single frame"""
        if self.model is None:
            return 0, []
        
        try:
            # Run YOLO detection - only detect people (class 0)
            results = self.model(frame, classes=[0], verbose=False)
            
            # Check if results are valid
            if not results or len(results) == 0 or results[0].boxes is None:
                return 0, []
            
            # Extract detection data
            boxes = results[0].boxes
            coords = boxes.xyxy.cpu().numpy()
            confs = boxes.conf.cpu().numpy()
            
            if len(coords) == 0:
                return 0, []
            
            # Create detection info
            people_detections = []
            for i in range(len(coords)):
                box = coords[i].astype(int)
                conf = float(confs[i])
                
                people_detections.append({
                    'box': box,
                    'confidence': conf,
                    'center': ((box[0] + box[2]) // 2, (box[1] + box[3]) // 2)
                })
            
            return len(people_detections), people_detections
            
        except Exception as e:
            print(f"⚠️ Detection error: {e}")
            return 0, []
    
    def draw_people_count_overlay(self, frame, people_count, people_detections):
        """Draw people count in left corner and bounding boxes"""
        annotated_frame = frame.copy()
        h, w = frame.shape[:2]
        
        # LEFT CORNER: Semi-transparent background for count display
        overlay = annotated_frame.copy()
        cv2.rectangle(overlay, (10, 10), (350, 140), (0, 0, 0), -1)
        cv2.addWeighted(overlay, 0.7, annotated_frame, 0.3, 0, annotated_frame)
        
        # Main people count display (LEFT CORNER)
        cv2.putText(annotated_frame, 'PEOPLE COUNT', (20, 40),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        cv2.putText(annotated_frame, str(people_count), (20, 85),
                   cv2.FONT_HERSHEY_SIMPLEX, 2.2, (0, 255, 0), 3)
        
        # Statistics in left corner
        if self.people_counts:
            max_people = max(self.people_counts)
            avg_people = np.mean(self.people_counts)
            cv2.putText(annotated_frame, f'Max: {max_people} | Avg: {avg_people:.1f}', 
                       (20, 125), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 200, 200), 2)
        
        # Draw bounding boxes around each detected person
        for detection in people_detections:
            box = detection['box']
            conf = detection['confidence']
            
            # Green rectangle around person
            cv2.rectangle(annotated_frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
            
            # Confidence score above box
            cv2.putText(annotated_frame, f'{conf:.2f}', (box[0], box[1]-8),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            
            # Center point
            center = detection['center']
            cv2.circle(annotated_frame, center, 4, (255, 0, 0), -1)
        
        # Model info (bottom)
        cv2.putText(annotated_frame, f'Model: {self.model_name}', 
                   (20, h - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        return annotated_frame
    
    def play_video_with_counting(self, video_path):
        """Play video with live people counting"""
        
        print(f"🎥 Opening video: {video_path}")
        
        # Open video file
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"❌ Cannot open video: {video_path}")
            print("💡 Make sure the file exists and is a valid video format (.mp4, .avi, .mov)")
            return
        
        # Get video properties
        fps = int(cap.get(cv2.CAP_PROP_FPS)) or 25
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        duration = total_frames / fps
        
        print(f"📹 Video Info: {width}x{height}, {fps} FPS, {total_frames} frames, {duration:.1f}s")
        print(f"🎬 Starting playback with people counting...")
        print("Press 'q' to quit, SPACE to pause/resume")
        
        # Reset statistics
        self.people_counts = []
        self.frame_count = 0
        
        frame_number = 0
        start_time = time.time()
        paused = False
        
        try:
            while True:
                if not paused:
                    ret, frame = cap.read()
                    if not ret:
                        print("📹 Video completed!")
                        break
                    
                    frame_number += 1
                    
                    # Count people in current frame
                    people_count, people_detections = self.count_people_in_frame(frame)
                    
                    # Update statistics
                    self.people_counts.append(people_count)
                    self.frame_count += 1
                    
                    # PRINT PEOPLE COUNT FOR EACH FRAME (as requested)
                    print(f"Frame {frame_number}: {people_count} people detected")
                    
                    # Draw count overlay and bounding boxes
                    display_frame = self.draw_people_count_overlay(frame, people_count, people_detections)
                    
                    # Add progress info
                    progress = (frame_number / total_frames) * 100
                    timestamp = frame_number / fps
                    cv2.putText(display_frame, f'Progress: {progress:.1f}%', 
                               (width - 200, height - 60),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                    cv2.putText(display_frame, f'Time: {timestamp:.1f}s', 
                               (width - 200, height - 30),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                
                # Display the frame
                cv2.imshow('Video Person Counter', display_frame)
                
                # Handle keyboard input
                key = cv2.waitKey(30) & 0xFF  # 30ms delay for smooth playback
                
                if key == ord('q') or key == 27:  # 'q' or ESC to quit
                    print("🛑 Stopped by user")
                    break
                elif key == ord(' '):  # SPACE to pause/resume
                    paused = not paused
                    print("⏸️ Paused" if paused else "▶️ Resumed")
                elif key == ord('s'):  # 's' to save screenshot
                    screenshot_name = f"screenshot_frame_{frame_number}.jpg"
                    cv2.imwrite(screenshot_name, display_frame)
                    print(f"📸 Screenshot saved: {screenshot_name}")
        
        except KeyboardInterrupt:
            print("\n🛑 Playback interrupted by user (Ctrl+C)")
        
        finally:
            # Cleanup
            cap.release()
            cv2.destroyAllWindows()
            
            # Final statistics
            total_time = time.time() - start_time
            
            if self.people_counts:
                print(f"\n📊 PLAYBACK COMPLETE:")
                print(f"   Frames processed: {self.frame_count}")
                print(f"   Playback time: {total_time:.1f}s")
                print(f"   Maximum people in frame: {max(self.people_counts)}")
                print(f"   Minimum people in frame: {min(self.people_counts)}")
                print(f"   Average people per frame: {np.mean(self.people_counts):.2f}")

# Initialize and use the counter
print("🚀 INITIALIZING VIDEO PERSON COUNTER")
print("=" * 50)

try:
    # Create counter instance
    counter = VideoPersonCounter('n')  # 'n' = nano (fastest), 's' = small, 'm' = medium
    
    print("\n✅ INITIALIZATION SUCCESSFUL!")
    print(f"📱 Model: {counter.model_name}")
    
    print(f"\n🎯 READY TO PLAY VIDEO WITH PEOPLE COUNTING!")
    print(f"\n🎬 CONTROLS:")
    print(f"   'q' or ESC = Quit")
    print(f"   SPACE = Pause/Resume")
    print(f"   's' = Save screenshot")
    
    # Play your video (replace with your video file path)
    counter.play_video_with_counting('testing8.mp4')
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("\n💡 TROUBLESHOOTING:")
    print("1. Install system dependencies: sudo apt-get install -y libgl1-mesa-glx libglib2.0-0")
    print("2. Install ultralytics: pip install ultralytics")
    print("3. Make sure video file exists and is in correct format")

print("\n🎉 READY TO USE!")


🚀 INITIALIZING VIDEO PERSON COUNTER
🚀 Initializing Video Person Counter...
📥 Trying to load yolo11n.pt...
✅ Successfully loaded yolo11n.pt

✅ INITIALIZATION SUCCESSFUL!
📱 Model: yolo11n.pt

🎯 READY TO PLAY VIDEO WITH PEOPLE COUNTING!

🎬 CONTROLS:
   'q' or ESC = Quit
   SPACE = Pause/Resume
   's' = Save screenshot
🎥 Opening video: testing8.mp4
📹 Video Info: 1920x1080, 30 FPS, 407 frames, 13.6s
🎬 Starting playback with people counting...
Press 'q' to quit, SPACE to pause/resume
Frame 1: 13 people detected
Frame 2: 12 people detected
Frame 3: 14 people detected
Frame 4: 14 people detected
Frame 5: 13 people detected
Frame 6: 15 people detected
Frame 7: 16 people detected
Frame 8: 13 people detected
Frame 9: 12 people detected
Frame 10: 9 people detected
Frame 11: 12 people detected
Frame 12: 11 people detected
Frame 13: 9 people detected
Frame 14: 9 people detected
Frame 15: 10 people detected
Frame 16: 10 people detected
Frame 17: 12 people detected
Frame 18: 12 people detected
Frame 1

In [1]:
import cv2
import numpy as np
import time
import warnings
warnings.filterwarnings('ignore')

class VideoPersonCounterFixed:
    def __init__(self, model_size='n'):
        print(f"🚀 Initializing Video Person Counter...")
        
        self.model = None
        self.model_name = "None"
        self.people_counts = []
        self.frame_count = 0
        self.display_method = "opencv"  # Will switch to matplotlib if needed
        
        # Test display capability
        self.test_display()
        
        # Load YOLO model
        self.load_model(model_size)
        
    def test_display(self):
        """Test if OpenCV display works"""
        try:
            test_img = np.zeros((100, 100, 3), dtype=np.uint8)
            cv2.imshow('test_window', test_img)
            cv2.waitKey(1)
            cv2.destroyWindow('test_window')
            self.display_method = "opencv"
            print("✅ OpenCV display available")
        except:
            self.display_method = "matplotlib"
            print("⚠️ OpenCV display not available, using matplotlib")
        
    def load_model(self, model_size):
        """Load YOLO model"""
        try:
            from ultralytics import YOLO
            
            model_options = [f'yolo11{model_size}.pt', f'yolov8{model_size}.pt', 'yolo11n.pt']
            
            for model_path in model_options:
                try:
                    print(f"📥 Loading {model_path}...")
                    self.model = YOLO(model_path)
                    self.model.conf = 0.5
                    self.model.iou = 0.45
                    self.model.max_det = 1000
                    self.model_name = model_path
                    print(f"✅ Successfully loaded {model_path}")
                    return
                except Exception as e:
                    continue
            
            print("❌ Failed to load YOLO model")
            
        except ImportError:
            print("❌ Ultralytics not installed")
    
    def count_people_in_frame(self, frame):
        """Count people in frame"""
        if self.model is None:
            return 0, []
        
        try:
            results = self.model(frame, classes=[0], verbose=False)  # Person class only
            
            if not results or len(results) == 0 or results[0].boxes is None:
                return 0, []
            
            boxes = results[0].boxes
            coords = boxes.xyxy.cpu().numpy()
            confs = boxes.conf.cpu().numpy()
            
            if len(coords) == 0:
                return 0, []
            
            people_detections = []
            for i in range(len(coords)):
                box = coords[i].astype(int)
                conf = float(confs[i])
                
                people_detections.append({
                    'box': box,
                    'confidence': conf,
                    'center': ((box[0] + box[2]) // 2, (box[1] + box[3]) // 2)
                })
            
            return len(people_detections), people_detections
            
        except Exception as e:
            print(f"⚠️ Detection error: {e}")
            return 0, []
    
    def draw_annotations(self, frame, people_count, people_detections):
        """Draw people count and bounding boxes"""
        annotated_frame = frame.copy()
        h, w = frame.shape[:2]
        
        # LEFT CORNER: Semi-transparent background for count display
        overlay = annotated_frame.copy()
        cv2.rectangle(overlay, (10, 10), (350, 140), (0, 0, 0), -1)
        cv2.addWeighted(overlay, 0.7, annotated_frame, 0.3, 0, annotated_frame)
        
        # Main people count display
        cv2.putText(annotated_frame, 'PEOPLE COUNT', (20, 40),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        cv2.putText(annotated_frame, str(people_count), (20, 85),
                   cv2.FONT_HERSHEY_SIMPLEX, 2.2, (0, 255, 0), 3)
        
        # Statistics
        if self.people_counts:
            max_people = max(self.people_counts)
            avg_people = np.mean(self.people_counts)
            cv2.putText(annotated_frame, f'Max: {max_people} | Avg: {avg_people:.1f}', 
                       (20, 125), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 200, 200), 2)
        
        # Draw bounding boxes around detected people
        for detection in people_detections:
            box = detection['box']
            conf = detection['confidence']
            
            # Green rectangle around person
            cv2.rectangle(annotated_frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
            
            # Confidence score
            cv2.putText(annotated_frame, f'{conf:.2f}', (box[0], box[1]-8),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            
            # Center point
            center = detection['center']
            cv2.circle(annotated_frame, center, 4, (255, 0, 0), -1)
        
        return annotated_frame
    
    def display_frame_safe(self, frame, frame_number):
        """Safe frame display with fallback to matplotlib"""
        
        if self.display_method == "opencv":
            try:
                cv2.imshow('Video Person Counter', frame)
                key = cv2.waitKey(30) & 0xFF
                return key
            except Exception as e:
                print(f"⚠️ OpenCV display failed: {e}")
                self.display_method = "matplotlib"
        
        # Matplotlib fallback (works everywhere)
        if self.display_method == "matplotlib":
            try:
                import matplotlib.pyplot as plt
                
                # Convert BGR to RGB for matplotlib
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                
                plt.figure(figsize=(12, 8))
                plt.imshow(frame_rgb)
                plt.title(f'Frame {frame_number} - Person Counter')
                plt.axis('off')
                plt.show()
                
                return ord('c')  # Continue
                
            except Exception as e:
                print(f"⚠️ Matplotlib display also failed: {e}")
                # Just continue without display
                return ord('c')
        
        return ord('c')
    
    def play_video_with_counting(self, video_path, display_video=True, save_frames=False):
        """Play video with people counting - handles display issues gracefully"""
        
        print(f"🎥 Opening video: {video_path}")
        
        # Open video
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"❌ Cannot open video: {video_path}")
            return
        
        # Get video properties
        fps = int(cap.get(cv2.CAP_PROP_FPS)) or 25
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        duration = total_frames / fps
        
        print(f"📹 Video Info: {width}x{height}, {fps} FPS, {total_frames} frames, {duration:.1f}s")
        print(f"🎬 Starting playback with people counting...")
        print(f"📺 Display method: {self.display_method}")
        
        # Reset statistics
        self.people_counts = []
        self.frame_count = 0
        
        frame_number = 0
        start_time = time.time()
        
        try:
            while True:
                ret, frame = cap.read()
                if not ret:
                    print("📹 Video completed!")
                    break
                
                frame_number += 1
                
                # Count people in current frame
                people_count, people_detections = self.count_people_in_frame(frame)
                
                # Update statistics
                self.people_counts.append(people_count)
                self.frame_count += 1
                
                # PRINT PEOPLE COUNT FOR EACH FRAME (as requested)
                print(f"Frame {frame_number}: {people_count} people detected")
                
                # Draw annotations
                display_frame = self.draw_annotations(frame, people_count, people_detections)
                
                # Add progress info
                progress = (frame_number / total_frames) * 100
                timestamp = frame_number / fps
                cv2.putText(display_frame, f'Progress: {progress:.1f}%', 
                           (width - 200, height - 60),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                cv2.putText(display_frame, f'Time: {timestamp:.1f}s', 
                           (width - 200, height - 30),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                
                # Display frame (handles both OpenCV and matplotlib)
                if display_video:
                    key = self.display_frame_safe(display_frame, frame_number)
                    
                    if key == ord('q') or key == 27:  # 'q' or ESC to quit
                        print("🛑 Stopped by user")
                        break
                
                # Save frames if requested
                if save_frames and frame_number % 30 == 0:  # Save every 30th frame
                    cv2.imwrite(f"frame_{frame_number:04d}_people_{people_count}.jpg", display_frame)
                
        except KeyboardInterrupt:
            print("\n🛑 Playback interrupted by user (Ctrl+C)")
        
        finally:
            # Safe cleanup
            cap.release()
            try:
                if self.display_method == "opencv":
                    cv2.destroyAllWindows()
            except:
                pass  # Ignore cleanup errors
            
            # Final statistics
            total_time = time.time() - start_time
            
            if self.people_counts:
                print(f"\n📊 PLAYBACK COMPLETE:")
                print(f"   Frames processed: {self.frame_count}")
                print(f"   Playback time: {total_time:.1f}s")
                print(f"   Maximum people in frame: {max(self.people_counts)}")
                print(f"   Minimum people in frame: {min(self.people_counts)}")
                print(f"   Average people per frame: {np.mean(self.people_counts):.2f}")
                print(f"   Display method used: {self.display_method}")

# Initialize and use the fixed counter
print("🚀 INITIALIZING FIXED VIDEO PERSON COUNTER")
print("=" * 55)

try:
    # Create counter with display fallback
    counter = VideoPersonCounterFixed('n')
    
    print("\n✅ INITIALIZATION SUCCESSFUL!")
    print(f"📱 Model: {counter.model_name}")
    print(f"📺 Display method: {counter.display_method}")
    
    # Play your video with safe display handling
    counter.play_video_with_counting('testing8.mp4', display_video=True)
    
except Exception as e:
    print(f"❌ Error: {e}")

print("\n🎉 COMPLETE!")


🚀 INITIALIZING FIXED VIDEO PERSON COUNTER
🚀 Initializing Video Person Counter...
✅ OpenCV display available
📥 Loading yolo11n.pt...
✅ Successfully loaded yolo11n.pt

✅ INITIALIZATION SUCCESSFUL!
📱 Model: yolo11n.pt
📺 Display method: opencv
🎥 Opening video: testing8.mp4
📹 Video Info: 1920x1080, 30 FPS, 407 frames, 13.6s
🎬 Starting playback with people counting...
📺 Display method: opencv
Frame 1: 13 people detected
Frame 2: 12 people detected
Frame 3: 14 people detected
Frame 4: 14 people detected
Frame 5: 13 people detected
Frame 6: 15 people detected
Frame 7: 16 people detected
Frame 8: 13 people detected
Frame 9: 12 people detected
Frame 10: 9 people detected
Frame 11: 12 people detected
Frame 12: 11 people detected
Frame 13: 9 people detected
Frame 14: 9 people detected
Frame 15: 10 people detected
Frame 16: 10 people detected
Frame 17: 12 people detected
Frame 18: 12 people detected
Frame 19: 12 people detected
Frame 20: 10 people detected
Frame 21: 11 people detected
Frame 22: 7 p

In [2]:
import cv2
import numpy as np
import time
import warnings
warnings.filterwarnings('ignore')

class VideoPersonCounter:
    def __init__(self, model_size='n'):
        """Initialize YOLO model for person counting"""
        print(f"🚀 Initializing Video Person Counter...")
        
        self.model = None
        self.model_name = "None"
        self.people_counts = []
        self.frame_count = 0
        
        # Screen dimensions (adjust these to your screen size)
        self.max_display_width = 1200   # Maximum window width
        self.max_display_height = 800   # Maximum window height
        
        # Load YOLO model
        self.load_model(model_size)
        
    def load_model(self, model_size):
        """Load YOLO model with error handling"""
        try:
            from ultralytics import YOLO
            
            # Try different model options
            model_options = [f'yolo11{model_size}.pt', f'yolov8{model_size}.pt', 'yolo11n.pt', 'yolov8n.pt']
            
            for model_path in model_options:
                try:
                    print(f"📥 Trying to load {model_path}...")
                    self.model = YOLO(model_path)
                    self.model.conf = 0.5       # Confidence threshold
                    self.model.iou = 0.45       # IoU threshold
                    self.model.max_det = 1000   # Max detections
                    self.model_name = model_path
                    print(f"✅ Successfully loaded {model_path}")
                    return
                except Exception as e:
                    print(f"⚠️ Failed to load {model_path}: {str(e)[:50]}...")
                    continue
            
            print("❌ All YOLO models failed to load")
            
        except ImportError:
            print("❌ Ultralytics not installed. Run: pip install ultralytics")
    
    def calculate_display_size(self, video_width, video_height):
        """Calculate optimal display size to show complete video"""
        
        # Calculate scaling factor to fit video within max display dimensions
        scale_width = self.max_display_width / video_width
        scale_height = self.max_display_height / video_height
        
        # Use the smaller scale to ensure the entire video fits
        scale_factor = min(scale_width, scale_height, 1.0)  # Don't scale up
        
        # Calculate new dimensions
        display_width = int(video_width * scale_factor)
        display_height = int(video_height * scale_factor)
        
        print(f"📺 Display scaling: {video_width}x{video_height} → {display_width}x{display_height} (scale: {scale_factor:.2f})")
        
        return display_width, display_height, scale_factor
    
    def count_people_in_frame(self, frame):
        """Count people in a single frame"""
        if self.model is None:
            return 0, []
        
        try:
            # Run YOLO detection - only detect people (class 0)
            results = self.model(frame, classes=[0], verbose=False)
            
            # Check if results are valid
            if not results or len(results) == 0 or results[0].boxes is None:
                return 0, []
            
            # Extract detection data
            boxes = results[0].boxes
            coords = boxes.xyxy.cpu().numpy()
            confs = boxes.conf.cpu().numpy()
            
            if len(coords) == 0:
                return 0, []
            
            # Create detection info
            people_detections = []
            for i in range(len(coords)):
                box = coords[i].astype(int)
                conf = float(confs[i])
                
                people_detections.append({
                    'box': box,
                    'confidence': conf,
                    'center': ((box[0] + box[2]) // 2, (box[1] + box[3]) // 2)
                })
            
            return len(people_detections), people_detections
            
        except Exception as e:
            print(f"⚠️ Detection error: {e}")
            return 0, []
    
    def draw_people_count_overlay(self, frame, people_count, people_detections, scale_factor=1.0):
        """Draw people count in left corner and bounding boxes with proper scaling"""
        annotated_frame = frame.copy()
        h, w = frame.shape[:2]
        
        # Scale elements based on display size
        font_scale = max(0.4, scale_factor * 0.8)
        thickness = max(1, int(scale_factor * 2))
        
        # LEFT CORNER: Semi-transparent background for count display
        overlay = annotated_frame.copy()
        rect_width = int(350 * scale_factor)
        rect_height = int(140 * scale_factor)
        cv2.rectangle(overlay, (10, 10), (10 + rect_width, 10 + rect_height), (0, 0, 0), -1)
        cv2.addWeighted(overlay, 0.7, annotated_frame, 0.3, 0, annotated_frame)
        
        # Main people count display (LEFT CORNER)
        cv2.putText(annotated_frame, 'PEOPLE COUNT', (20, int(40 * scale_factor)),
                   cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 255, 255), thickness)
        cv2.putText(annotated_frame, str(people_count), (20, int(85 * scale_factor)),
                   cv2.FONT_HERSHEY_SIMPLEX, font_scale * 2.5, (0, 255, 0), thickness + 1)
        
        # Statistics in left corner
        if self.people_counts:
            max_people = max(self.people_counts)
            avg_people = np.mean(self.people_counts)
            cv2.putText(annotated_frame, f'Max: {max_people} | Avg: {avg_people:.1f}', 
                       (20, int(125 * scale_factor)), cv2.FONT_HERSHEY_SIMPLEX, font_scale * 0.8, (200, 200, 200), thickness)
        
        # Draw bounding boxes around each detected person
        for detection in people_detections:
            box = detection['box']
            conf = detection['confidence']
            
            # Green rectangle around person
            cv2.rectangle(annotated_frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), thickness)
            
            # Confidence score above box
            cv2.putText(annotated_frame, f'{conf:.2f}', (box[0], box[1]-8),
                       cv2.FONT_HERSHEY_SIMPLEX, font_scale * 0.7, (0, 255, 0), thickness)
            
            # Center point
            center = detection['center']
            cv2.circle(annotated_frame, center, max(2, int(4 * scale_factor)), (255, 0, 0), -1)
        
        # Model info (bottom)
        cv2.putText(annotated_frame, f'Model: {self.model_name}', 
                   (20, h - int(20 * scale_factor)), cv2.FONT_HERSHEY_SIMPLEX, font_scale * 0.8, (255, 255, 255), thickness)
        
        return annotated_frame
    
    def play_video_with_counting(self, video_path):
        """Play video with live people counting - COMPLETE VIDEO DISPLAY"""
        
        print(f"🎥 Opening video: {video_path}")
        
        # Open video file
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"❌ Cannot open video: {video_path}")
            print("💡 Make sure the file exists and is a valid video format (.mp4, .avi, .mov)")
            return
        
        # Get video properties
        fps = int(cap.get(cv2.CAP_PROP_FPS)) or 25
        original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        duration = total_frames / fps
        
        # Calculate display dimensions to show complete video
        display_width, display_height, scale_factor = self.calculate_display_size(
            original_width, original_height
        )
        
        print(f"📹 Video Info: {original_width}x{original_height}, {fps} FPS, {total_frames} frames, {duration:.1f}s")
        print(f"🎬 Starting playback with complete video display...")
        print("Press 'q' to quit, SPACE to pause/resume, 'f' for fullscreen")
        
        # Create resizable window for complete video display
        cv2.namedWindow('Complete Video Person Counter', cv2.WINDOW_NORMAL)
        cv2.resizeWindow('Complete Video Person Counter', display_width, display_height)
        
        # Reset statistics
        self.people_counts = []
        self.frame_count = 0
        
        frame_number = 0
        start_time = time.time()
        paused = False
        
        try:
            while True:
                if not paused:
                    ret, frame = cap.read()
                    if not ret:
                        print("📹 Video completed!")
                        break
                    
                    frame_number += 1
                    
                    # Count people in ORIGINAL frame (better accuracy)
                    people_count, people_detections = self.count_people_in_frame(frame)
                    
                    # Update statistics
                    self.people_counts.append(people_count)
                    self.frame_count += 1
                    
                    # PRINT PEOPLE COUNT FOR EACH FRAME (as requested)
                    print(f"Frame {frame_number}: {people_count} people detected")
                    
                    # Draw count overlay and bounding boxes on original frame
                    display_frame = self.draw_people_count_overlay(frame, people_count, people_detections, scale_factor)
                    
                    # Add progress info (scaled for original resolution)
                    progress = (frame_number / total_frames) * 100
                    timestamp = frame_number / fps
                    cv2.putText(display_frame, f'Progress: {progress:.1f}%', 
                               (original_width - int(200 * scale_factor), original_height - int(60 * scale_factor)),
                               cv2.FONT_HERSHEY_SIMPLEX, scale_factor * 0.6, (255, 255, 255), max(1, int(scale_factor * 2)))
                    cv2.putText(display_frame, f'Time: {timestamp:.1f}s', 
                               (original_width - int(200 * scale_factor), original_height - int(30 * scale_factor)),
                               cv2.FONT_HERSHEY_SIMPLEX, scale_factor * 0.6, (255, 255, 255), max(1, int(scale_factor * 2)))
                    
                    # Resize frame for display (this shows the COMPLETE video)
                    resized_display_frame = cv2.resize(display_frame, (display_width, display_height))
                
                # Display the COMPLETE resized frame
                cv2.imshow('Complete Video Person Counter', resized_display_frame)
                
                # Handle keyboard input
                key = cv2.waitKey(max(1, int(1000/fps))) & 0xFF  # Match video FPS
                
                if key == ord('q') or key == 27:  # 'q' or ESC to quit
                    print("🛑 Stopped by user")
                    break
                elif key == ord(' '):  # SPACE to pause/resume
                    paused = not paused
                    print("⏸️ Paused" if paused else "▶️ Resumed")
                elif key == ord('f'):  # 'f' for fullscreen
                    cv2.setWindowProperty('Complete Video Person Counter', 
                                        cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
                    print("🔍 Fullscreen mode")
                elif key == ord('n'):  # 'n' for normal window
                    cv2.setWindowProperty('Complete Video Person Counter', 
                                        cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_NORMAL)
                    print("📺 Normal window mode")
                elif key == ord('s'):  # 's' to save screenshot
                    screenshot_name = f"screenshot_frame_{frame_number}.jpg"
                    cv2.imwrite(screenshot_name, resized_display_frame)
                    print(f"📸 Screenshot saved: {screenshot_name}")
        
        except KeyboardInterrupt:
            print("\n🛑 Playback interrupted by user (Ctrl+C)")
        
        finally:
            # Cleanup
            cap.release()
            cv2.destroyAllWindows()
            
            # Final statistics
            total_time = time.time() - start_time
            
            if self.people_counts:
                print(f"\n📊 PLAYBACK COMPLETE:")
                print(f"   Original video: {original_width}x{original_height}")
                print(f"   Display size: {display_width}x{display_height}")
                print(f"   Scale factor: {scale_factor:.2f}")
                print(f"   Frames processed: {self.frame_count}")
                print(f"   Playback time: {total_time:.1f}s")
                print(f"   Maximum people in frame: {max(self.people_counts)}")
                print(f"   Minimum people in frame: {min(self.people_counts)}")
                print(f"   Average people per frame: {np.mean(self.people_counts):.2f}")

# Initialize and use the counter
print("🚀 INITIALIZING COMPLETE VIDEO PERSON COUNTER")
print("=" * 55)

try:
    # Create counter instance
    counter = VideoPersonCounter('n')  # 'n' = nano (fastest), 's' = small, 'm' = medium
    
    # Adjust these values to match your screen size
    counter.max_display_width = 1200    # Change to fit your screen
    counter.max_display_height = 800    # Change to fit your screen
    
    print("\n✅ INITIALIZATION SUCCESSFUL!")
    print(f"📱 Model: {counter.model_name}")
    print(f"🖥️ Max display size: {counter.max_display_width}x{counter.max_display_height}")
    
    print(f"\n🎯 READY TO PLAY COMPLETE VIDEO WITH PEOPLE COUNTING!")
    print(f"\n🎬 CONTROLS:")
    print(f"   'q' or ESC = Quit")
    print(f"   SPACE = Pause/Resume")
    print(f"   'f' = Fullscreen")
    print(f"   'n' = Normal window")
    print(f"   's' = Save screenshot")
    
    # Play your video (replace with your video file path)
    counter.play_video_with_counting('testing8.mp4')
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("\n💡 TROUBLESHOOTING:")
    print("1. Install system dependencies: sudo apt-get install -y libgl1-mesa-glx libglib2.0-0")
    print("2. Install ultralytics: pip install ultralytics")
    print("3. Make sure video file exists and is in correct format")

print("\n🎉 COMPLETE VIDEO DISPLAY READY!")


🚀 INITIALIZING COMPLETE VIDEO PERSON COUNTER
🚀 Initializing Video Person Counter...
📥 Trying to load yolo11n.pt...
✅ Successfully loaded yolo11n.pt

✅ INITIALIZATION SUCCESSFUL!
📱 Model: yolo11n.pt
🖥️ Max display size: 1200x800

🎯 READY TO PLAY COMPLETE VIDEO WITH PEOPLE COUNTING!

🎬 CONTROLS:
   'q' or ESC = Quit
   SPACE = Pause/Resume
   'f' = Fullscreen
   'n' = Normal window
   's' = Save screenshot
🎥 Opening video: testing8.mp4
📺 Display scaling: 1920x1080 → 1200x675 (scale: 0.62)
📹 Video Info: 1920x1080, 30 FPS, 407 frames, 13.6s
🎬 Starting playback with complete video display...
Press 'q' to quit, SPACE to pause/resume, 'f' for fullscreen
Frame 1: 13 people detected
Frame 2: 12 people detected
Frame 3: 14 people detected
Frame 4: 14 people detected
Frame 5: 13 people detected
Frame 6: 15 people detected
Frame 7: 16 people detected
Frame 8: 13 people detected
Frame 9: 12 people detected
Frame 10: 9 people detected
Frame 11: 12 people detected
Frame 12: 11 people detected
Frame 1

In [2]:
import cv2
import numpy as np
import time
import warnings
warnings.filterwarnings('ignore')

class LineBasedPersonCounter:
    def __init__(self, model_size='n', line_position_ratio=0.85):
        """
        Initialize line-based person counter
        model_size: YOLO model size ('n', 's', 'm', 'l', 'x')
        line_position_ratio: Line position as ratio of frame height (0.0 = top, 1.0 = bottom)
        """
        print(f"🚀 Initializing Line-Based Person Counter...")
        
        self.model = None
        self.model_name = "None"
        self.line_position_ratio = line_position_ratio
        
        # Counting variables
        self.total_people_counted = 0
        self.frame_count = 0
        self.people_per_frame = []
        
        # Load YOLO model
        self.load_model(model_size)
        
    def load_model(self, model_size):
        """Load YOLO model with error handling"""
        try:
            from ultralytics import YOLO
            
            model_options = [f'yolo11{model_size}.pt', f'yolov8{model_size}.pt', 'yolo11n.pt', 'yolov8n.pt']
            
            for model_path in model_options:
                try:
                    print(f"📥 Trying to load {model_path}...")
                    self.model = YOLO(model_path)
                    self.model.conf = 0.5       # Confidence threshold
                    self.model.iou = 0.45       # IoU threshold
                    self.model.max_det = 1000   # Max detections
                    self.model_name = model_path
                    print(f"✅ Successfully loaded {model_path}")
                    return
                except Exception as e:
                    print(f"⚠️ Failed to load {model_path}: {str(e)[:50]}...")
                    continue
            
            print("❌ All YOLO models failed to load")
            
        except ImportError:
            print("❌ Ultralytics not installed. Run: pip install ultralytics")
    
    def detect_people_in_frame(self, frame):
        """Detect people in frame and return bounding boxes"""
        if self.model is None:
            return []
        
        try:
            # Run YOLO detection - only detect people (class 0)
            results = self.model(frame, classes=[0], verbose=False)
            
            people_boxes = []
            
            if results and len(results) > 0 and results[0].boxes is not None:
                boxes = results[0].boxes
                coords = boxes.xyxy.cpu().numpy()
                confs = boxes.conf.cpu().numpy()
                
                for i in range(len(coords)):
                    box = coords[i].astype(int)
                    conf = float(confs[i])
                    
                    people_boxes.append({
                        'box': box,  # [x1, y1, x2, y2]
                        'confidence': conf,
                        'center': ((box[0] + box[2]) // 2, (box[1] + box[3]) // 2),
                        'bottom_y': box[3]  # Bottom of bounding box
                    })
            
            return people_boxes
            
        except Exception as e:
            print(f"⚠️ Detection error: {e}")
            return []
    
    def count_people_crossing_line(self, people_boxes, line_y):
        """Count people whose bounding boxes touch or cross the counting line"""
        crossing_count = 0
        
        for person in people_boxes:
            bottom_y = person['bottom_y']
            
            # Count if person's bottom edge touches or crosses the line
            if bottom_y >= line_y:
                crossing_count += 1
        
        return crossing_count
    
    def calculate_display_size(self, video_width, video_height):
        """Calculate display size to fit screen"""
        max_width = 1200
        max_height = 800
        
        scale_width = max_width / video_width
        scale_height = max_height / video_height
        scale_factor = min(scale_width, scale_height, 1.0)
        
        display_width = int(video_width * scale_factor)
        display_height = int(video_height * scale_factor)
        
        return display_width, display_height, scale_factor
    
    def draw_line_and_annotations(self, frame, people_boxes, line_y, crossing_count, scale_factor=1.0):
        """Draw counting line, bounding boxes, and count display"""
        annotated_frame = frame.copy()
        h, w = frame.shape[:2]
        
        # Scale elements for display
        font_scale = max(0.6, scale_factor * 0.8)
        thickness = max(2, int(scale_factor * 3))
        
        # Draw the GREEN COUNTING LINE (horizontal)
        cv2.line(annotated_frame, (0, line_y), (w, line_y), (0, 255, 0), thickness + 2)
        
        # Add line label
        cv2.putText(annotated_frame, 'COUNTING LINE', (w // 2 - 100, line_y - 10),
                   cv2.FONT_HERSHEY_SIMPLEX, font_scale * 0.7, (0, 255, 0), thickness)
        
        # Draw bounding boxes around detected people
        for person in people_boxes:
            box = person['box']
            conf = person['confidence']
            bottom_y = person['bottom_y']
            
            # Color: RED if crossing line, BLUE if not crossing
            if bottom_y >= line_y:
                color = (0, 0, 255)  # Red - crossing line
                label = f"CROSSING {conf:.2f}"
            else:
                color = (255, 0, 0)  # Blue - not crossing
                label = f"{conf:.2f}"
            
            # Draw bounding box
            cv2.rectangle(annotated_frame, (box[0], box[1]), (box[2], box[3]), color, thickness)
            
            # Draw confidence and status
            cv2.putText(annotated_frame, label, (box[0], box[1] - 8),
                       cv2.FONT_HERSHEY_SIMPLEX, font_scale * 0.5, color, thickness - 1)
            
            # Draw center point
            center = person['center']
            cv2.circle(annotated_frame, center, max(3, int(5 * scale_factor)), color, -1)
        
        # LEFT CORNER: Count display with background
        overlay = annotated_frame.copy()
        cv2.rectangle(overlay, (10, 10), (400, 180), (0, 0, 0), -1)
        cv2.addWeighted(overlay, 0.7, annotated_frame, 0.3, 0, annotated_frame)
        
        # Current frame crossing count
        cv2.putText(annotated_frame, 'PEOPLE CROSSING LINE', (20, 40),
                   cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 255, 255), thickness)
        cv2.putText(annotated_frame, f'This Frame: {crossing_count}', (20, 80),
                   cv2.FONT_HERSHEY_SIMPLEX, font_scale * 1.2, (0, 255, 255), thickness)
        
        # Total count for entire video
        cv2.putText(annotated_frame, f'TOTAL COUNT: {self.total_people_counted}', (20, 130),
                   cv2.FONT_HERSHEY_SIMPLEX, font_scale * 1.2, (0, 255, 0), thickness + 1)
        
        # Statistics
        if self.people_per_frame:
            avg_per_frame = np.mean(self.people_per_frame)
            cv2.putText(annotated_frame, f'Avg/Frame: {avg_per_frame:.1f}', (20, 165),
                       cv2.FONT_HERSHEY_SIMPLEX, font_scale * 0.7, (200, 200, 200), thickness - 1)
        
        return annotated_frame
    
    def play_video_with_line_counting(self, video_path):
        """Play video with line-based people counting"""
        
        print(f"🎥 Opening video: {video_path}")
        
        # Open video
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"❌ Cannot open video: {video_path}")
            return
        
        # Get video properties
        fps = int(cap.get(cv2.CAP_PROP_FPS)) or 25
        original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        duration = total_frames / fps
        
        # Calculate counting line position
        line_y = int(original_height * self.line_position_ratio)
        
        # Calculate display size
        display_width, display_height, scale_factor = self.calculate_display_size(
            original_width, original_height
        )
        
        print(f"📹 Video Info: {original_width}x{original_height}, {fps} FPS, {total_frames} frames, {duration:.1f}s")
        print(f"📏 Counting line at Y = {line_y} ({self.line_position_ratio * 100:.1f}% from top)")
        print(f"🎬 Starting line-based counting...")
        
        # Create window
        cv2.namedWindow('Line-Based Person Counter', cv2.WINDOW_NORMAL)
        cv2.resizeWindow('Line-Based Person Counter', display_width, display_height)
        
        # Reset counters
        self.total_people_counted = 0
        self.frame_count = 0
        self.people_per_frame = []
        
        frame_number = 0
        start_time = time.time()
        paused = False
        
        print("Press 'q' to quit, SPACE to pause/resume, 'f' for fullscreen")
        
        try:
            while True:
                if not paused:
                    ret, frame = cap.read()
                    if not ret:
                        print("📹 Video completed!")
                        break
                    
                    frame_number += 5
                    self.frame_count += 5
                    
                    # Detect people in current frame
                    people_boxes = self.detect_people_in_frame(frame)
                    
                    # Count people crossing the line in this frame
                    crossing_count = self.count_people_crossing_line(people_boxes, line_y)
                    
                    # Add to total count
                    self.total_people_counted += crossing_count
                    self.people_per_frame.append(crossing_count)
                    
                    # PRINT COUNT FOR EACH FRAME
                    print(f"Frame {frame_number}: {crossing_count} people crossing line | Total: {self.total_people_counted}")
                    
                    # Draw annotations
                    display_frame = self.draw_line_and_annotations(
                        frame, people_boxes, line_y, crossing_count, scale_factor
                    )
                    
                    # Add progress info
                    progress = (frame_number / total_frames) * 100
                    timestamp = frame_number / fps
                    cv2.putText(display_frame, f'Progress: {progress:.1f}% | Time: {timestamp:.1f}s', 
                               (original_width - int(300 * scale_factor), original_height - int(30 * scale_factor)),
                               cv2.FONT_HERSHEY_SIMPLEX, scale_factor * 0.6, (255, 255, 255), max(1, int(scale_factor * 2)))
                    
                    # Resize for display
                    resized_frame = cv2.resize(display_frame, (display_width, display_height))
                
                # Display frame
                cv2.imshow('Line-Based Person Counter', resized_frame)
                
                # Handle keyboard input
                key = cv2.waitKey(max(1, int(1000/fps))) & 0xFF
                
                if key == ord('q') or key == 27:  # 'q' or ESC
                    print("🛑 Stopped by user")
                    break
                elif key == ord(' '):  # SPACE to pause/resume
                    paused = not paused
                    print("⏸️ Paused" if paused else "▶️ Resumed")
                elif key == ord('f'):  # 'f' for fullscreen
                    cv2.setWindowProperty('Line-Based Person Counter', 
                                        cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
                elif key == ord('n'):  # 'n' for normal
                    cv2.setWindowProperty('Line-Based Person Counter', 
                                        cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_NORMAL)
                elif key == ord('s'):  # 's' to save screenshot
                    screenshot_name = f"line_counter_frame_{frame_number}.jpg"
                    cv2.imwrite(screenshot_name, resized_frame)
                    print(f"📸 Screenshot saved: {screenshot_name}")
        
        except KeyboardInterrupt:
            print("\n🛑 Playback interrupted by user (Ctrl+C)")
        
        finally:
            # Cleanup
            cap.release()
            cv2.destroyAllWindows()
            
            # Final statistics
            total_time = time.time() - start_time
            
            print(f"\n📊 LINE-BASED COUNTING COMPLETE:")
            print(f"   📏 Counting line position: Y = {line_y} ({self.line_position_ratio * 100:.1f}% from top)")
            print(f"   🎯 Total people counted crossing line: {self.total_people_counted}")
            print(f"   📹 Frames processed: {self.frame_count}")
            print(f"   ⏱️ Processing time: {total_time:.1f}s")
            if self.people_per_frame:
                print(f"   📈 Maximum crossings in single frame: {max(self.people_per_frame)}")
                print(f"   📊 Average crossings per frame: {np.mean(self.people_per_frame):.2f}")
            print(f"   🎥 Video: {original_width}x{original_height}, Line at Y={line_y}")

# Initialize and use the line-based counter
print("🚀 INITIALIZING LINE-BASED PERSON COUNTER")
print("=" * 60)

try:
    # Create line-based counter
    counter = LineBasedPersonCounter('n', line_position_ratio=0.90)  # Line at 85% from top
    
    print("\n✅ INITIALIZATION SUCCESSFUL!")
    print(f"📱 Model: {counter.model_name}")
    print(f"📏 Line position: {counter.line_position_ratio * 100:.1f}% from top")
    
    print(f"\n🎯 READY FOR LINE-BASED COUNTING!")
    print(f"\n🎬 CONTROLS:")
    print(f"   'q' or ESC = Quit")
    print(f"   SPACE = Pause/Resume")
    print(f"   'f' = Fullscreen")
    print(f"   'n' = Normal window")
    print(f"   's' = Save screenshot")
    
    print(f"\n📏 HOW IT WORKS:")
    print(f"   • Green horizontal line appears at {counter.line_position_ratio * 100:.1f}% from top")
    print(f"   • People are counted ONLY when they touch/cross this line")
    print(f"   • Red boxes = people crossing line, Blue boxes = people not crossing")
    print(f"   • Total count accumulates for entire video")
    
    # Play your video with line-based counting
    counter.play_video_with_line_counting('testing10.mp4')
    
except Exception as e:
    print(f"❌ Error: {e}")

print("\n🎉 LINE-BASED COUNTING COMPLETE!")


🚀 INITIALIZING LINE-BASED PERSON COUNTER
🚀 Initializing Line-Based Person Counter...
📥 Trying to load yolo11n.pt...
✅ Successfully loaded yolo11n.pt

✅ INITIALIZATION SUCCESSFUL!
📱 Model: yolo11n.pt
📏 Line position: 90.0% from top

🎯 READY FOR LINE-BASED COUNTING!

🎬 CONTROLS:
   'q' or ESC = Quit
   SPACE = Pause/Resume
   'f' = Fullscreen
   'n' = Normal window
   's' = Save screenshot

📏 HOW IT WORKS:
   • Green horizontal line appears at 90.0% from top
   • People are counted ONLY when they touch/cross this line
   • Red boxes = people crossing line, Blue boxes = people not crossing
   • Total count accumulates for entire video
🎥 Opening video: testing10.mp4
📹 Video Info: 768x432, 25 FPS, 750 frames, 30.0s
📏 Counting line at Y = 388 (90.0% from top)
🎬 Starting line-based counting...
Press 'q' to quit, SPACE to pause/resume, 'f' for fullscreen
Frame 5: 5 people crossing line | Total: 5
Frame 10: 2 people crossing line | Total: 7
Frame 15: 1 people crossing line | Total: 8
Frame 20: 

In [4]:
import cv2
import numpy as np
import time
import warnings
from collections import OrderedDict
from scipy.spatial import distance as dist
warnings.filterwarnings('ignore')

class CentroidTracker:
    def __init__(self, maxDisappeared=30, maxDistance=80):
        """Simple centroid tracker for person movement tracking"""
        self.nextObjectID = 0
        self.objects = OrderedDict()
        self.disappeared = OrderedDict()
        self.maxDisappeared = maxDisappeared
        self.maxDistance = maxDistance
        
        # For direction tracking
        self.centroid_history = OrderedDict()  # Store recent positions
        self.max_history_length = 10

    def register(self, centroid):
        """Register new tracked object"""
        self.objects[self.nextObjectID] = centroid
        self.disappeared[self.nextObjectID] = 0
        self.centroid_history[self.nextObjectID] = [centroid]
        self.nextObjectID += 1

    def deregister(self, objectID):
        """Remove tracked object"""
        del self.objects[objectID]
        del self.disappeared[objectID]
        del self.centroid_history[objectID]

    def update(self, rects):
        """Update tracker with new detections"""
        if len(rects) == 0:
            # Mark all existing objects as disappeared
            for objectID in list(self.disappeared.keys()):
                self.disappeared[objectID] += 1
                if self.disappeared[objectID] > self.maxDisappeared:
                    self.deregister(objectID)
            return self.objects

        # Initialize centroids for current frame
        inputCentroids = np.zeros((len(rects), 2), dtype="int")
        for (i, (startX, startY, endX, endY)) in enumerate(rects):
            cX = int((startX + endX) / 2.0)
            cY = int((startY + endY) / 2.0)
            inputCentroids[i] = (cX, cY)

        # If no existing objects, register all as new
        if len(self.objects) == 0:
            for i in range(0, len(inputCentroids)):
                self.register(inputCentroids[i])
        else:
            # Match existing objects to new detections
            objectIDs = list(self.objects.keys())
            objectCentroids = list(self.objects.values())

            # Compute distance matrix
            D = dist.cdist(np.array(objectCentroids), inputCentroids)
            
            # Find minimum distances
            rows = D.min(axis=1).argsort()
            cols = D.argmin(axis=1)[rows]

            usedRows = set()
            usedCols = set()

            # Update matched objects
            for (row, col) in zip(rows, cols):
                if row in usedRows or col in usedCols:
                    continue

                if D[row, col] > self.maxDistance:
                    continue

                objectID = objectIDs[row]
                self.objects[objectID] = inputCentroids[col]
                self.disappeared[objectID] = 0

                # Update centroid history
                self.centroid_history[objectID].append(tuple(inputCentroids[col]))
                if len(self.centroid_history[objectID]) > self.max_history_length:
                    self.centroid_history[objectID].pop(0)

                usedRows.add(row)
                usedCols.add(col)

            # Handle unmatched objects and detections
            unusedRows = set(range(0, D.shape[0])).difference(usedRows)
            unusedCols = set(range(0, D.shape[1])).difference(usedCols)

            # Mark unmatched objects as disappeared
            for row in unusedRows:
                objectID = objectIDs[row]
                self.disappeared[objectID] += 1
                if self.disappeared[objectID] > self.maxDisappeared:
                    self.deregister(objectID)

            # Register new objects
            for col in unusedCols:
                self.register(inputCentroids[col])

        return self.objects

class DirectionalPersonCounter:
    def __init__(self, model_size='n', line_position_ratio=0.85):
        """
        Direction-aware person counter with tracking
        Only counts people moving from bottom to top, ignores stationary people
        """
        print(f"🚀 Initializing Directional Person Counter...")
        
        self.model = None
        self.model_name = "None"
        self.line_position_ratio = line_position_ratio
        
        # Initialize tracker
        self.tracker = CentroidTracker(maxDisappeared=30, maxDistance=100)
        
        # Counting variables
        self.total_people_counted = 0  # Net count (up - down)
        self.people_entered = 0        # Bottom to top crossings
        self.people_exited = 0         # Top to bottom crossings
        self.frame_count = 0
        
        # Track line crossings
        self.line_crossed_objects = {}  # objectID: {'crossed': bool, 'direction': str, 'position': str}
        self.movement_threshold = 5     # Minimum pixels to consider as movement
        
        # Load YOLO model
        self.load_model(model_size)
        
    def load_model(self, model_size):
        """Load YOLO model with error handling"""
        try:
            from ultralytics import YOLO
            
            model_options = [f'yolo11{model_size}.pt', f'yolov8{model_size}.pt', 'yolo11n.pt', 'yolov8n.pt']
            
            for model_path in model_options:
                try:
                    print(f"📥 Trying to load {model_path}...")
                    self.model = YOLO(model_path)
                    self.model.conf = 0.5
                    self.model.iou = 0.45
                    self.model.max_det = 1000
                    self.model_name = model_path
                    print(f"✅ Successfully loaded {model_path}")
                    return
                except Exception as e:
                    continue
            
            print("❌ All YOLO models failed to load")
            
        except ImportError:
            print("❌ Ultralytics not installed")
    
    def detect_people_in_frame(self, frame):
        """Detect people and return bounding boxes"""
        if self.model is None:
            return []
        
        try:
            results = self.model(frame, classes=[0], verbose=False)
            people_boxes = []
            
            if results and len(results) > 0 and results[0].boxes is not None:
                boxes = results[0].boxes
                coords = boxes.xyxy.cpu().numpy()
                confs = boxes.conf.cpu().numpy()
                
                for i in range(len(coords)):
                    box = coords[i].astype(int)
                    conf = float(confs[i])
                    
                    people_boxes.append([box[0], box[1], box[2], box[3]])  # [x1, y1, x2, y2]
            
            return people_boxes
            
        except Exception as e:
            print(f"⚠️ Detection error: {e}")
            return []
    
    def calculate_movement_direction(self, objectID, line_y):
        """Calculate movement direction for tracked object"""
        if objectID not in self.tracker.centroid_history:
            return None, False
        
        history = self.tracker.centroid_history[objectID]
        if len(history) < 3:  # Need at least 3 points for reliable direction
            return None, False
        
        # Get recent positions
        recent_positions = history[-5:]  # Last 5 positions
        
        # Calculate overall vertical movement
        start_y = recent_positions[0][1]
        end_y = recent_positions[-1][1]
        vertical_movement = end_y - start_y
        
        # Check if movement is significant (not stationary)
        if abs(vertical_movement) < self.movement_threshold:
            return "stationary", False
        
        # Determine direction
        if vertical_movement > 0:
            return "top_to_bottom", True  # Moving down
        else:
            return "bottom_to_top", True  # Moving up
    
    def check_line_crossings(self, objects, line_y):
        """Check for line crossings and update counts"""
        crossings_this_frame = []
        
        for objectID, centroid in objects.items():
            current_y = centroid[1]
            
            # Initialize tracking for new objects
            if objectID not in self.line_crossed_objects:
                position = "below" if current_y > line_y else "above"
                self.line_crossed_objects[objectID] = {
                    'crossed': False,
                    'last_position': position,
                    'current_position': position
                }
            
            # Update current position
            current_position = "below" if current_y > line_y else "above"
            last_position = self.line_crossed_objects[objectID]['last_position']
            
            # Check for line crossing
            if last_position != current_position:
                direction, is_moving = self.calculate_movement_direction(objectID, line_y)
                
                if is_moving and direction != "stationary":
                    if last_position == "below" and current_position == "above":
                        # Person moved from bottom to top (ENTERING)
                        if direction == "bottom_to_top":
                            self.people_entered += 1
                            self.total_people_counted += 1
                            crossings_this_frame.append({
                                'objectID': objectID,
                                'direction': 'ENTERING',
                                'type': 'bottom_to_top',
                                'centroid': centroid
                            })
                            print(f"✅ Person {objectID} ENTERED (bottom→top) | Total: {self.total_people_counted}")
                    
                    elif last_position == "above" and current_position == "below":
                        # Person moved from top to bottom (EXITING)
                        if direction == "top_to_bottom":
                            self.people_exited += 1
                            self.total_people_counted -= 1
                            crossings_this_frame.append({
                                'objectID': objectID,
                                'direction': 'EXITING',
                                'type': 'top_to_bottom',
                                'centroid': centroid
                            })
                            print(f"❌ Person {objectID} EXITED (top→bottom) | Total: {self.total_people_counted}")
            
            # Update position tracking
            self.line_crossed_objects[objectID]['last_position'] = current_position
            self.line_crossed_objects[objectID]['current_position'] = current_position
        
        # Clean up tracking for disappeared objects
        current_object_ids = set(objects.keys())
        to_remove = []
        for objectID in self.line_crossed_objects.keys():
            if objectID not in current_object_ids:
                to_remove.append(objectID)
        
        for objectID in to_remove:
            del self.line_crossed_objects[objectID]
        
        return crossings_this_frame
    
    def calculate_display_size(self, video_width, video_height):
        """Calculate display size to fit screen"""
        max_width = 1200
        max_height = 800
        
        scale_width = max_width / video_width
        scale_height = max_height / video_height
        scale_factor = min(scale_width, scale_height, 1.0)
        
        display_width = int(video_width * scale_factor)
        display_height = int(video_height * scale_factor)
        
        return display_width, display_height, scale_factor
    
    def draw_tracking_and_counting(self, frame, objects, line_y, crossings, scale_factor=1.0):
        """Draw tracking, line, and counting information"""
        annotated_frame = frame.copy()
        h, w = frame.shape[:2]
        
        font_scale = max(0.6, scale_factor * 0.8)
        thickness = max(2, int(scale_factor * 3))
        
        # Draw the GREEN COUNTING LINE
        cv2.line(annotated_frame, (0, line_y), (w, line_y), (0, 255, 0), thickness + 2)
        cv2.putText(annotated_frame, 'DIRECTIONAL COUNTING LINE', (w // 2 - 150, line_y - 15),
                   cv2.FONT_HERSHEY_SIMPLEX, font_scale * 0.7, (0, 255, 0), thickness)
        
        # Draw tracked objects with direction info
        for objectID, centroid in objects.items():
            direction, is_moving = self.calculate_movement_direction(objectID, line_y)
            
            # Color coding based on movement and position
            if direction == "stationary":
                color = (128, 128, 128)  # Gray for stationary
                status = "STATIONARY"
            elif direction == "bottom_to_top":
                color = (0, 255, 0)      # Green for upward movement
                status = "MOVING UP"
            elif direction == "top_to_bottom":
                color = (0, 0, 255)      # Red for downward movement
                status = "MOVING DOWN"
            else:
                color = (255, 255, 255)  # White for unknown
                status = "TRACKING"
            
            # Draw tracking circle
            cv2.circle(annotated_frame, centroid, 8, color, -1)
            cv2.circle(annotated_frame, centroid, 12, color, 2)
            
            # Draw object ID and status
            cv2.putText(annotated_frame, f"ID:{objectID}", 
                       (centroid[0] - 20, centroid[1] - 25),
                       cv2.FONT_HERSHEY_SIMPLEX, font_scale * 0.5, color, thickness - 1)
            cv2.putText(annotated_frame, status, 
                       (centroid[0] - 30, centroid[1] + 25),
                       cv2.FONT_HERSHEY_SIMPLEX, font_scale * 0.4, color, thickness - 1)
            
            # Draw movement trail
            if objectID in self.tracker.centroid_history:
                history = self.tracker.centroid_history[objectID]
                if len(history) > 1:
                    for i in range(1, len(history)):
                        cv2.line(annotated_frame, history[i-1], history[i], color, 2)
        
        # Highlight recent crossings
        for crossing in crossings:
            centroid = crossing['centroid']
            if crossing['direction'] == 'ENTERING':
                cv2.circle(annotated_frame, centroid, 20, (0, 255, 0), 3)
                cv2.putText(annotated_frame, "ENTERED!", (centroid[0] - 30, centroid[1] - 30),
                           cv2.FONT_HERSHEY_SIMPLEX, font_scale * 0.6, (0, 255, 0), thickness)
            else:
                cv2.circle(annotated_frame, centroid, 20, (0, 0, 255), 3)
                cv2.putText(annotated_frame, "EXITED!", (centroid[0] - 25, centroid[1] - 30),
                           cv2.FONT_HERSHEY_SIMPLEX, font_scale * 0.6, (0, 0, 255), thickness)
        
        # LEFT CORNER: Counting display
        overlay = annotated_frame.copy()
        cv2.rectangle(overlay, (10, 10), (450, 220), (0, 0, 0), -1)
        cv2.addWeighted(overlay, 0.8, annotated_frame, 0.2, 0, annotated_frame)
        
        # Count information
        cv2.putText(annotated_frame, 'DIRECTIONAL COUNTER', (20, 35),
                   cv2.FONT_HERSHEY_SIMPLEX, font_scale * 0.9, (255, 255, 255), thickness)
        
        cv2.putText(annotated_frame, f'NET COUNT: {self.total_people_counted}', (20, 70),
                   cv2.FONT_HERSHEY_SIMPLEX, font_scale * 1.2, (0, 255, 0), thickness + 1)
        
        cv2.putText(annotated_frame, f'Entered (↑): {self.people_entered}', (20, 105),
                   cv2.FONT_HERSHEY_SIMPLEX, font_scale * 0.8, (0, 255, 0), thickness)
        cv2.putText(annotated_frame, f'Exited (↓): {self.people_exited}', (20, 135),
                   cv2.FONT_HERSHEY_SIMPLEX, font_scale * 0.8, (0, 0, 255), thickness)
        
        cv2.putText(annotated_frame, f'Active Tracks: {len(objects)}', (20, 165),
                   cv2.FONT_HERSHEY_SIMPLEX, font_scale * 0.7, (255, 255, 0), thickness - 1)
        
        # Legend
        cv2.putText(annotated_frame, 'Green=Up, Red=Down, Gray=Still', (20, 195),
                   cv2.FONT_HERSHEY_SIMPLEX, font_scale * 0.6, (200, 200, 200), thickness - 1)
        
        return annotated_frame
    
    def play_video_with_directional_counting(self, video_path):
        """Play video with directional people counting"""
        
        print(f"🎥 Opening video: {video_path}")
        
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"❌ Cannot open video: {video_path}")
            return
        
        # Get video properties
        fps = int(cap.get(cv2.CAP_PROP_FPS)) or 25
        original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        duration = total_frames / fps
        
        line_y = int(original_height * self.line_position_ratio)
        display_width, display_height, scale_factor = self.calculate_display_size(
            original_width, original_height
        )
        
        print(f"📹 Video: {original_width}x{original_height}, {fps} FPS, {total_frames} frames")
        print(f"📏 Counting line at Y = {line_y} ({self.line_position_ratio * 100:.1f}% from top)")
        print(f"🎯 Direction: Only counting bottom→top (entering), subtracting top→bottom (exiting)")
        
        cv2.namedWindow('Directional Person Counter', cv2.WINDOW_NORMAL)
        cv2.resizeWindow('Directional Person Counter', display_width, display_height)
        
        frame_number = 0
        start_time = time.time()
        paused = False
        
        print("🎬 Controls: 'q'=quit, SPACE=pause/resume, 'f'=fullscreen")
        
        try:
            while True:
                if not paused:
                    ret, frame = cap.read()
                    if not ret:
                        print("📹 Video completed!")
                        break
                    
                    frame_number += 1
                    self.frame_count += 1
                    
                    # Detect people
                    people_boxes = self.detect_people_in_frame(frame)
                    
                    # Update tracker
                    objects = self.tracker.update(people_boxes)
                    
                    # Check for line crossings and update counts
                    crossings = self.check_line_crossings(objects, line_y)
                    
                    # Draw everything
                    display_frame = self.draw_tracking_and_counting(
                        frame, objects, line_y, crossings, scale_factor
                    )
                    
                    # Add progress info
                    progress = (frame_number / total_frames) * 100
                    timestamp = frame_number / fps
                    cv2.putText(display_frame, f'Progress: {progress:.1f}% | Time: {timestamp:.1f}s', 
                               (original_width - int(300 * scale_factor), original_height - int(30 * scale_factor)),
                               cv2.FONT_HERSHEY_SIMPLEX, scale_factor * 0.6, (255, 255, 255), max(1, int(scale_factor * 2)))
                    
                    resized_frame = cv2.resize(display_frame, (display_width, display_height))
                
                cv2.imshow('Directional Person Counter', resized_frame)
                
                key = cv2.waitKey(max(1, int(1000/fps))) & 0xFF
                
                if key == ord('q') or key == 27:
                    print("🛑 Stopped by user")
                    break
                elif key == ord(' '):
                    paused = not paused
                    print("⏸️ Paused" if paused else "▶️ Resumed")
                elif key == ord('f'):
                    cv2.setWindowProperty('Directional Person Counter', 
                                        cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
                elif key == ord('n'):
                    cv2.setWindowProperty('Directional Person Counter', 
                                        cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_NORMAL)
        
        except KeyboardInterrupt:
            print("\n🛑 Interrupted by user")
        
        finally:
            cap.release()
            cv2.destroyAllWindows()
            
            processing_time = time.time() - start_time
            
            print(f"\n📊 DIRECTIONAL COUNTING COMPLETE:")
            print(f"   📏 Line position: Y = {line_y} ({self.line_position_ratio * 100:.1f}% from top)")
            print(f"   🎯 NET COUNT (current occupancy): {self.total_people_counted}")
            print(f"   ⬆️ Total entered (bottom→top): {self.people_entered}")
            print(f"   ⬇️ Total exited (top→bottom): {self.people_exited}")
            print(f"   📹 Frames processed: {self.frame_count}")
            print(f"   ⏱️ Processing time: {processing_time:.1f}s")
            print(f"   🎥 Video: {original_width}x{original_height}")

# Install required package first (run once)
# pip install scipy

# Initialize and use the directional counter
print("🚀 INITIALIZING DIRECTIONAL PERSON COUNTER")
print("=" * 65)

try:
    # Create directional counter
    counter = DirectionalPersonCounter('n', line_position_ratio=0.90)
    
    print("\n✅ INITIALIZATION SUCCESSFUL!")
    print(f"📱 Model: {counter.model_name}")
    print(f"📏 Line position: {counter.line_position_ratio * 100:.1f}% from top")
    
    print(f"\n🎯 DIRECTIONAL COUNTING FEATURES:")
    print(f"   ✅ Only counts people moving bottom→top (entering)")
    print(f"   ❌ Subtracts count for people moving top→bottom (exiting)")
    print(f"   🚫 Ignores stationary people")
    print(f"   🔍 Tracks individual people across frames")
    print(f"   📊 Shows net occupancy count")
    
    print(f"\n🎬 CONTROLS:")
    print(f"   'q' or ESC = Quit")
    print(f"   SPACE = Pause/Resume")
    print(f"   'f' = Fullscreen")
    print(f"   'n' = Normal window")
    
    print(f"\n🎨 VISUAL INDICATORS:")
    print(f"   🟢 Green circles = People moving up (entering)")
    print(f"   🔴 Red circles = People moving down (exiting)")
    print(f"   ⚫ Gray circles = Stationary people (ignored)")
    print(f"   🔍 Trails show movement paths")
    
    # Run with your video
    counter.play_video_with_directional_counting('testing10.mp4')
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("\n💡 REQUIREMENTS:")
    print("   pip install ultralytics opencv-python scipy")

print("\n🎉 DIRECTIONAL COUNTING READY!")


🚀 INITIALIZING DIRECTIONAL PERSON COUNTER
🚀 Initializing Directional Person Counter...
📥 Trying to load yolo11n.pt...
✅ Successfully loaded yolo11n.pt

✅ INITIALIZATION SUCCESSFUL!
📱 Model: yolo11n.pt
📏 Line position: 90.0% from top

🎯 DIRECTIONAL COUNTING FEATURES:
   ✅ Only counts people moving bottom→top (entering)
   ❌ Subtracts count for people moving top→bottom (exiting)
   🚫 Ignores stationary people
   🔍 Tracks individual people across frames
   📊 Shows net occupancy count

🎬 CONTROLS:
   'q' or ESC = Quit
   SPACE = Pause/Resume
   'f' = Fullscreen
   'n' = Normal window

🎨 VISUAL INDICATORS:
   🟢 Green circles = People moving up (entering)
   🔴 Red circles = People moving down (exiting)
   ⚫ Gray circles = Stationary people (ignored)
   🔍 Trails show movement paths
🎥 Opening video: testing10.mp4
📹 Video: 768x432, 25 FPS, 750 frames
📏 Counting line at Y = 388 (90.0% from top)
🎯 Direction: Only counting bottom→top (entering), subtracting top→bottom (exiting)
🎬 Controls: 'q'=qui

In [6]:
import cv2
import numpy as np
import time
import warnings
from collections import OrderedDict
from scipy.spatial import distance as dist
warnings.filterwarnings('ignore')

class CentroidTracker:
    def __init__(self, maxDisappeared=30, maxDistance=80):
        """Simple centroid tracker for person movement tracking"""
        self.nextObjectID = 0
        self.objects = OrderedDict()
        self.disappeared = OrderedDict()
        self.maxDisappeared = maxDisappeared
        self.maxDistance = maxDistance
        
        # For direction tracking
        self.centroid_history = OrderedDict()
        self.max_history_length = 8

    def register(self, centroid):
        """Register new tracked object"""
        self.objects[self.nextObjectID] = centroid
        self.disappeared[self.nextObjectID] = 0
        self.centroid_history[self.nextObjectID] = [centroid]
        self.nextObjectID += 1

    def deregister(self, objectID):
        """Remove tracked object"""
        del self.objects[objectID]
        del self.disappeared[objectID]
        del self.centroid_history[objectID]

    def update(self, rects):
        """Update tracker with new detections"""
        if len(rects) == 0:
            for objectID in list(self.disappeared.keys()):
                self.disappeared[objectID] += 1
                if self.disappeared[objectID] > self.maxDisappeared:
                    self.deregister(objectID)
            return self.objects

        inputCentroids = np.zeros((len(rects), 2), dtype="int")
        for (i, (startX, startY, endX, endY)) in enumerate(rects):
            cX = int((startX + endX) / 2.0)
            cY = int((startY + endY) / 2.0)
            inputCentroids[i] = (cX, cY)

        if len(self.objects) == 0:
            for i in range(len(inputCentroids)):
                self.register(inputCentroids[i])
        else:
            objectIDs = list(self.objects.keys())
            objectCentroids = list(self.objects.values())

            D = dist.cdist(np.array(objectCentroids), inputCentroids)
            rows = D.min(axis=1).argsort()
            cols = D.argmin(axis=1)[rows]

            usedRows = set()
            usedCols = set()

            for (row, col) in zip(rows, cols):
                if row in usedRows or col in usedCols:
                    continue
                if D[row, col] > self.maxDistance:
                    continue

                objectID = objectIDs[row]
                self.objects[objectID] = inputCentroids[col]
                self.disappeared[objectID] = 0

                # Update centroid history
                self.centroid_history[objectID].append(tuple(inputCentroids[col]))
                if len(self.centroid_history[objectID]) > self.max_history_length:
                    self.centroid_history[objectID].pop(0)

                usedRows.add(row)
                usedCols.add(col)

            # Handle unmatched objects and detections
            unusedRows = set(range(0, D.shape[0])).difference(usedRows)
            unusedCols = set(range(0, D.shape[1])).difference(usedCols)

            for row in unusedRows:
                objectID = objectIDs[row]
                self.disappeared[objectID] += 1
                if self.disappeared[objectID] > self.maxDisappeared:
                    self.deregister(objectID)

            for col in unusedCols:
                self.register(inputCentroids[col])

        return self.objects

class EnteringOnlyPersonCounter:
    def __init__(self, model_size='n', line_position_ratio=0.85):
        """
        Person counter that ONLY tracks and displays people entering (bottom→top)
        Completely ignores exiting and stationary people
        """
        print(f"🚀 Initializing ENTERING-ONLY Person Counter...")
        
        self.model = None
        self.model_name = "None"
        self.line_position_ratio = line_position_ratio
        
        # Initialize tracker
        self.tracker = CentroidTracker(maxDisappeared=25, maxDistance=100)
        
        # Counting variables - ONLY for entering people
        self.total_entered_count = 0
        self.frame_count = 0
        
        # Track only entering people
        self.entered_people_ids = set()  # IDs of people who have entered
        self.movement_threshold = 8      # Minimum pixels to consider as movement
        
        # Load YOLO model
        self.load_model(model_size)
        
    def load_model(self, model_size):
        """Load YOLO model with error handling"""
        try:
            from ultralytics import YOLO
            
            model_options = [f'yolo11{model_size}.pt', f'yolov8{model_size}.pt', 'yolo11n.pt', 'yolov8n.pt']
            
            for model_path in model_options:
                try:
                    print(f"📥 Trying to load {model_path}...")
                    self.model = YOLO(model_path)
                    self.model.conf = 0.5
                    self.model.iou = 0.45
                    self.model.max_det = 1000
                    self.model_name = model_path
                    print(f"✅ Successfully loaded {model_path}")
                    return
                except Exception as e:
                    continue
            
            print("❌ All YOLO models failed to load")
            
        except ImportError:
            print("❌ Ultralytics not installed")
    
    def detect_people_in_frame(self, frame):
        """Detect people and return bounding boxes"""
        if self.model is None:
            return []
        
        try:
            results = self.model(frame, classes=[0], verbose=False)
            people_boxes = []
            
            if results and len(results) > 0 and results[0].boxes is not None:
                boxes = results[0].boxes
                coords = boxes.xyxy.cpu().numpy()
                confs = boxes.conf.cpu().numpy()
                
                for i in range(len(coords)):
                    box = coords[i].astype(int)
                    conf = float(confs[i])
                    people_boxes.append([box[0], box[1], box[2], box[3]])
            
            return people_boxes
            
        except Exception as e:
            print(f"⚠️ Detection error: {e}")
            return []
    
    def get_movement_direction(self, objectID):
        """Calculate movement direction for tracked object"""
        if objectID not in self.tracker.centroid_history:
            return None, False
        
        history = self.tracker.centroid_history[objectID]
        if len(history) < 4:  # Need enough points for reliable direction
            return None, False
        
        # Get recent positions for direction calculation
        recent_positions = history[-4:]
        
        # Calculate overall vertical movement
        start_y = recent_positions[0][1]
        end_y = recent_positions[-1][1]
        vertical_movement = end_y - start_y
        
        # Check if movement is significant (not stationary)
        if abs(vertical_movement) < self.movement_threshold:
            return "stationary", False
        
        # Determine direction
        if vertical_movement > 0:
            return "moving_down", True  # Top to bottom (exiting)
        else:
            return "moving_up", True    # Bottom to top (entering)
    
    def filter_entering_people_only(self, objects, line_y):
        """Filter to show ONLY people who are entering (moving bottom→top)"""
        entering_people = {}
        new_entries_this_frame = []
        
        for objectID, centroid in objects.items():
            current_y = centroid[1]
            
            # Get movement direction
            direction, is_moving = self.get_movement_direction(objectID)
            
            # ONLY process people moving UP (entering)
            if is_moving and direction == "moving_up":
                
                # Check if this person just crossed the line (entered)
                if objectID in self.tracker.centroid_history and len(self.tracker.centroid_history[objectID]) >= 2:
                    prev_y = self.tracker.centroid_history[objectID][-2][1]
                    
                    # Check for line crossing from bottom to top
                    if prev_y > line_y and current_y <= line_y:
                        if objectID not in self.entered_people_ids:
                            self.total_entered_count += 1
                            self.entered_people_ids.add(objectID)
                            new_entries_this_frame.append({
                                'objectID': objectID,
                                'centroid': centroid,
                                'just_entered': True
                            })
                            print(f"✅ Person {objectID} ENTERED! Total entered: {self.total_entered_count}")
                
                # Add to display list (only entering people)
                entering_people[objectID] = {
                    'centroid': centroid,
                    'direction': direction,
                    'is_entered': objectID in self.entered_people_ids,
                    'just_entered': any(entry['objectID'] == objectID for entry in new_entries_this_frame)
                }
            
            # COMPLETELY IGNORE:
            # - People moving down (exiting)
            # - Stationary people
            # - People not moving significantly
        
        return entering_people, new_entries_this_frame
    
    def calculate_display_size(self, video_width, video_height):
        """Calculate display size to fit screen"""
        max_width = 1200
        max_height = 800
        
        scale_width = max_width / video_width
        scale_height = max_height / video_height
        scale_factor = min(scale_width, scale_height, 1.0)
        
        display_width = int(video_width * scale_factor)
        display_height = int(video_height * scale_factor)
        
        return display_width, display_height, scale_factor
    
    def draw_entering_people_only(self, frame, entering_people, new_entries, line_y, scale_factor=1.0):
        """Draw ONLY entering people and their tracking"""
        annotated_frame = frame.copy()
        h, w = frame.shape[:2]
        
        font_scale = max(0.6, scale_factor * 0.8)
        thickness = max(2, int(scale_factor * 3))
        
        # Draw the GREEN COUNTING LINE
        cv2.line(annotated_frame, (0, line_y), (w, line_y), (0, 255, 0), thickness + 2)
        cv2.putText(annotated_frame, 'ENTRY LINE (BOTTOM→TOP ONLY)', (w // 2 - 200, line_y - 15),
                   cv2.FONT_HERSHEY_SIMPLEX, font_scale * 0.7, (0, 255, 0), thickness)
        
        # Draw ONLY entering people
        for objectID, person_data in entering_people.items():
            centroid = person_data['centroid']
            is_entered = person_data['is_entered']
            just_entered = person_data['just_entered']
            
            # Color coding for entering people only
            if just_entered:
                color = (0, 255, 255)  # Yellow for just entered
                status = "JUST ENTERED!"
                circle_radius = 15
            elif is_entered:
                color = (0, 255, 0)    # Green for previously entered
                status = "ENTERED"
                circle_radius = 10
            else:
                color = (0, 200, 255)  # Orange for approaching
                status = "APPROACHING"
                circle_radius = 8
            
            # Draw tracking circle
            cv2.circle(annotated_frame, centroid, circle_radius, color, -1)
            cv2.circle(annotated_frame, centroid, circle_radius + 3, color, 2)
            
            # Draw object ID and status
            cv2.putText(annotated_frame, f"ID:{objectID}", 
                       (centroid[0] - 20, centroid[1] - 30),
                       cv2.FONT_HERSHEY_SIMPLEX, font_scale * 0.6, color, thickness)
            cv2.putText(annotated_frame, status, 
                       (centroid[0] - 40, centroid[1] + 35),
                       cv2.FONT_HERSHEY_SIMPLEX, font_scale * 0.5, color, thickness)
            
            # Draw movement trail for entering people
            if objectID in self.tracker.centroid_history:
                history = self.tracker.centroid_history[objectID]
                if len(history) > 1:
                    for i in range(1, len(history)):
                        cv2.line(annotated_frame, history[i-1], history[i], color, 3)
        
        # Highlight new entries with animation effect
        for entry in new_entries:
            centroid = entry['centroid']
            cv2.circle(annotated_frame, centroid, 25, (0, 255, 255), 4)
            cv2.putText(annotated_frame, "NEW ENTRY!", (centroid[0] - 50, centroid[1] - 40),
                       cv2.FONT_HERSHEY_SIMPLEX, font_scale * 0.8, (0, 255, 255), thickness + 1)
        
        # LEFT CORNER: Entry-only count display
        overlay = annotated_frame.copy()
        cv2.rectangle(overlay, (10, 10), (400, 160), (0, 0, 0), -1)
        cv2.addWeighted(overlay, 0.8, annotated_frame, 0.2, 0, annotated_frame)
        
        # Count information - ONLY entries
        cv2.putText(annotated_frame, 'ENTERING PEOPLE ONLY', (20, 35),
                   cv2.FONT_HERSHEY_SIMPLEX, font_scale * 0.9, (255, 255, 255), thickness)
        
        cv2.putText(annotated_frame, f'TOTAL ENTERED: {self.total_entered_count}', (20, 75),
                   cv2.FONT_HERSHEY_SIMPLEX, font_scale * 1.3, (0, 255, 0), thickness + 1)
        
        cv2.putText(annotated_frame, f'Currently Tracking: {len(entering_people)}', (20, 110),
                   cv2.FONT_HERSHEY_SIMPLEX, font_scale * 0.8, (0, 255, 255), thickness)
        
        # Legend - ONLY for entering people
        cv2.putText(annotated_frame, 'Yellow=Just Entered, Green=Entered, Orange=Approaching', (20, 140),
                   cv2.FONT_HERSHEY_SIMPLEX, font_scale * 0.5, (200, 200, 200), thickness - 1)
        
        return annotated_frame
    
    def play_video_entering_only(self, video_path):
        """Play video showing ONLY entering people"""
        
        print(f"🎥 Opening video: {video_path}")
        
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"❌ Cannot open video: {video_path}")
            return
        
        # Get video properties
        fps = int(cap.get(cv2.CAP_PROP_FPS)) or 25
        original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        duration = total_frames / fps
        
        line_y = int(original_height * self.line_position_ratio)
        display_width, display_height, scale_factor = self.calculate_display_size(
            original_width, original_height
        )
        
        print(f"📹 Video: {original_width}x{original_height}, {fps} FPS, {total_frames} frames")
        print(f"📏 Entry line at Y = {line_y} ({self.line_position_ratio * 100:.1f}% from top)")
        print(f"🎯 Mode: ENTERING PEOPLE ONLY (bottom→top)")
        print(f"🚫 Ignoring: Exiting people (top→bottom) and stationary people")
        
        cv2.namedWindow('Entering People Only Counter', cv2.WINDOW_NORMAL)
        cv2.resizeWindow('Entering People Only Counter', display_width, display_height)
        
        frame_number = 0
        start_time = time.time()
        paused = False
        
        print("🎬 Controls: 'q'=quit, SPACE=pause/resume, 'f'=fullscreen")
        
        try:
            while True:
                if not paused:
                    ret, frame = cap.read()
                    if not ret:
                        print("📹 Video completed!")
                        break
                    
                    frame_number += 1
                    self.frame_count += 1
                    
                    # Detect people
                    people_boxes = self.detect_people_in_frame(frame)
                    
                    # Update tracker
                    objects = self.tracker.update(people_boxes)
                    
                    # Filter to show ONLY entering people
                    entering_people, new_entries = self.filter_entering_people_only(objects, line_y)
                    
                    # Print frame info - ONLY for entering people
                    if entering_people or new_entries:
                        print(f"Frame {frame_number}: {len(entering_people)} entering people tracked | Total entered: {self.total_entered_count}")
                    
                    # Draw ONLY entering people
                    display_frame = self.draw_entering_people_only(
                        frame, entering_people, new_entries, line_y, scale_factor
                    )
                    
                    # Add progress info
                    progress = (frame_number / total_frames) * 100
                    timestamp = frame_number / fps
                    cv2.putText(display_frame, f'Progress: {progress:.1f}% | Time: {timestamp:.1f}s', 
                               (original_width - int(300 * scale_factor), original_height - int(30 * scale_factor)),
                               cv2.FONT_HERSHEY_SIMPLEX, scale_factor * 0.6, (255, 255, 255), max(1, int(scale_factor * 2)))
                    
                    resized_frame = cv2.resize(display_frame, (display_width, display_height))
                
                cv2.imshow('Entering People Only Counter', resized_frame)
                
                key = cv2.waitKey(max(1, int(1000/fps))) & 0xFF
                
                if key == ord('q') or key == 27:
                    print("🛑 Stopped by user")
                    break
                elif key == ord(' '):
                    paused = not paused
                    print("⏸️ Paused" if paused else "▶️ Resumed")
                elif key == ord('f'):
                    cv2.setWindowProperty('Entering People Only Counter', 
                                        cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
                elif key == ord('n'):
                    cv2.setWindowProperty('Entering People Only Counter', 
                                        cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_NORMAL)
        
        except KeyboardInterrupt:
            print("\n🛑 Interrupted by user")
        
        finally:
            cap.release()
            cv2.destroyAllWindows()
            
            processing_time = time.time() - start_time
            
            print(f"\n📊 ENTERING-ONLY COUNTING COMPLETE:")
            print(f"   📏 Entry line position: Y = {line_y} ({self.line_position_ratio * 100:.1f}% from top)")
            print(f"   ✅ TOTAL PEOPLE ENTERED: {self.total_entered_count}")
            print(f"   📹 Frames processed: {self.frame_count}")
            print(f"   ⏱️ Processing time: {processing_time:.1f}s")
            print(f"   🎯 Mode: ENTERING ONLY (ignored exiting & stationary people)")

# Install required packages (run once)
# pip install ultralytics opencv-python scipy

# Initialize and use the entering-only counter
print("🚀 INITIALIZING ENTERING-ONLY PERSON COUNTER")
print("=" * 70)

try:
    # Create entering-only counter
    counter = EnteringOnlyPersonCounter('n', line_position_ratio=0.90)
    
    print("\n✅ INITIALIZATION SUCCESSFUL!")
    print(f"📱 Model: {counter.model_name}")
    print(f"📏 Entry line position: {counter.line_position_ratio * 100:.1f}% from top")
    
    print(f"\n🎯 ENTERING-ONLY FEATURES:")
    print(f"   ✅ Only tracks people moving bottom→top (entering)")
    print(f"   🚫 Completely ignores people moving top→bottom (exiting)")
    print(f"   🚫 Completely ignores stationary people")
    print(f"   🔍 Only displays entering people with trails")
    print(f"   📊 Shows only total entered count (no subtraction)")
    
    print(f"\n🎬 CONTROLS:")
    print(f"   'q' or ESC = Quit")
    print(f"   SPACE = Pause/Resume")
    print(f"   'f' = Fullscreen")
    print(f"   'n' = Normal window")
    
    print(f"\n🎨 VISUAL INDICATORS (ENTERING PEOPLE ONLY):")
    print(f"   🟡 Yellow circles = Just entered through line")
    print(f"   🟢 Green circles = Previously entered people")
    print(f"   🟠 Orange circles = Approaching entry line")
    print(f"   🔍 Colored trails = Movement paths of entering people")
    
    # Run with your video
    counter.play_video_entering_only('testing9.mp4')
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("\n💡 REQUIREMENTS:")
    print("   pip install ultralytics opencv-python scipy")

print("\n🎉 ENTERING-ONLY COUNTING READY!")


🚀 INITIALIZING ENTERING-ONLY PERSON COUNTER
🚀 Initializing ENTERING-ONLY Person Counter...
📥 Trying to load yolo11n.pt...
✅ Successfully loaded yolo11n.pt

✅ INITIALIZATION SUCCESSFUL!
📱 Model: yolo11n.pt
📏 Entry line position: 90.0% from top

🎯 ENTERING-ONLY FEATURES:
   ✅ Only tracks people moving bottom→top (entering)
   🚫 Completely ignores people moving top→bottom (exiting)
   🚫 Completely ignores stationary people
   🔍 Only displays entering people with trails
   📊 Shows only total entered count (no subtraction)

🎬 CONTROLS:
   'q' or ESC = Quit
   SPACE = Pause/Resume
   'f' = Fullscreen
   'n' = Normal window

🎨 VISUAL INDICATORS (ENTERING PEOPLE ONLY):
   🟡 Yellow circles = Just entered through line
   🟢 Green circles = Previously entered people
   🟠 Orange circles = Approaching entry line
   🔍 Colored trails = Movement paths of entering people
🎥 Opening video: testing9.mp4
📹 Video: 768x432, 30 FPS, 169 frames
📏 Entry line at Y = 388 (90.0% from top)
🎯 Mode: ENTERING PEOPLE ON